# Analysis List Moves

Let's try to do something with our list of moves ...

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

In [2]:
data = pd.read_csv('../data_othello.csv')
display(data.head())
input_shape = data.shape
print("Input shape : {}".format(input_shape))

FileNotFoundError: [Errno 2] No such file or directory: './data_othello.csv'

This time, the only thing that matter is the liste of moves and the win (for the label) so let go for some data transformation !

In [36]:
def split_lst_move(row) :

    row['Liste Moves'] = row['Liste Moves'].replace('[', '')
    row['Liste Moves'] = row['Liste Moves'].replace(']', '')
    row['Liste Moves'] = row['Liste Moves'].replace('((', '')
    l_row = row['Liste Moves'].split("), ")
    l_row[-1] = l_row[-1][:-1]
    # print(row['win'])
    # row = row.drop(['Liste Moves'], axis = 1)
    if len(l_row) == 120 : # Drop party that ended before filled the whole board (don't know how to do for the density map otherwise)
        for i in range(0,len(l_row)-1, 2):
            if (int(l_row[i + 1]) < 0 and int(row['Score']) < 0) or (int(l_row[i + 1]) > 0 and int(row['Score']) > 0) :
                n_row = l_row[i].split(", ")
                row[str(round(i/2) ) + 'x'] = n_row[0]
                row[str(round(i/2) ) + 'y'] = n_row[1]
                # row[str(round(i/2)) + 'p'] = l_row[i + 1]
    return row[1:]

# row1 = df.iloc[1]
# print(split_lst_move(row1))

In [37]:
df = data.drop(data.iloc[:, :2].columns, axis=1)
df = df.drop(df.iloc[:,2:].columns, axis=1)
# df = df[:500] # Work on a smaller part for try
df = df.apply(lambda row: split_lst_move(row), axis = 1)
df.head()
# pd.DataFrame(df['Liste Moves'].tolist(), index=df.index)
# df.head()

,0x,0y,10x,10y,11x,11y,12x,12y,13x,13y,...,5y,6x,6y,7x,7y,8x,8y,9x,9y,Score
0,NaN,NaN,NaN,NaN,4,2,NaN,NaN,0,2,...,1,NaN,NaN,2,6,NaN,NaN,0,6,-12
1,2,3,5,4,NaN,NaN,4,1,NaN,NaN,...,NaN,0,3,NaN,NaN,5,2,NaN,NaN,10
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,NaN,NaN,NaN,NaN,2,1,NaN,NaN,1,1,...,4,NaN,NaN,3,1,NaN,NaN,5,4,-6
4,NaN,NaN,NaN,NaN,3,1,NaN,NaN,2,3,...,5,NaN,NaN,6,3,NaN,NaN,2,4,-4


In [39]:
def generate_z(row, Z):
    row = row.dropna()
    for i in range(0, len(row) - 2, 2):
        Z[int(row[i])][int(row[i+1])] += 1

In [42]:
# Sample data
side = np.linspace(0,7,8)
X,Y = np.meshgrid(side,side)
Z = np.zeros((8,8))
df.apply(lambda row: generate_z(row, Z), axis = 1)

# Plot the density map using nearest-neighbor interpolation
# fig = plt.pcolormesh(X,Y,Z)
# plt.title('Density Map')
# plt.show()
# fig = fig.get_figure()
# fig.savefig('./images/Density_Map.png')

0        None
1        None
2        None
3        None
4        None
         ... 
16199    None
16200    None
16201    None
16202    None
16203    None
Length: 16204, dtype: object

This density map was generated by keeping only the moves played by the winning player. Most of the dataset was generated using randomness and MCTS. MinMax and Alpha-Beta using a density map doesn't influence to much our result.

<img src="./images/Density_Map.png">


In [15]:
def split_lst_move2(row) :
    row['Liste Moves'] = row['Liste Moves'].replace('[', '')
    row['Liste Moves'] = row['Liste Moves'].replace(']', '')
    row['Liste Moves'] = row['Liste Moves'].replace('((', '')
    l_row = row['Liste Moves'].split("), ")
    l_row[-1] = l_row[-1][:-1]
    for i in range(0,len(l_row)-1, 2):
        row[str(l_row[i])] = int(i/2)
    return row[1:]

In [16]:
df2 = data.drop(data.iloc[:, :2].columns, axis=1)
df2 = df2.drop(df2.iloc[:,2:].columns, axis=1)
df2 = df2.apply(lambda row: split_lst_move2(row), axis = 1)
df2.head()

,"0, 0","0, 1","0, 2","0, 3","0, 4","0, 5","0, 6","0, 7","1, 0","1, 1",...,"6, 7","7, 0","7, 1","7, 2","7, 3","7, 4","7, 5","7, 6","7, 7",Score
0,25.0,27.0,13.0,44.0,55.0,39.0,9.0,23.0,59.0,6.0,...,58.0,47.0,40.0,37.0,34.0,53.0,42.0,54.0,46.0,-12
1,13.0,4.0,19.0,6.0,9.0,14.0,31.0,56.0,30.0,3.0,...,50.0,41.0,59.0,27.0,37.0,48.0,40.0,43.0,46.0,10
2,57.0,56.0,46.0,51.0,18.0,55.0,22.0,58.0,50.0,23.0,...,47.0,34.0,41.0,42.0,45.0,10.0,11.0,30.0,40.0,0
3,40.0,26.0,18.0,32.0,38.0,59.0,58.0,43.0,29.0,13.0,...,22.0,41.0,36.0,35.0,46.0,48.0,31.0,57.0,47.0,-6
4,52.0,41.0,22.0,17.0,32.0,47.0,39.0,40.0,53.0,51.0,...,33.0,55.0,27.0,23.0,8.0,31.0,48.0,58.0,56.0,-4


In [38]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

The idea is to see if it is possible to predict who wins with all the moves played in order. For the moment I test without indicating who has played the moves but just the order. 

In [25]:
df_test = df2.copy()

df_train = df_test.sample(frac = 0.7, random_state = 0)
df_valid = df_test.drop(df_train.index)

X_train = df_train.drop('Score', axis = 1)
X_valid = df_valid.drop('Score', axis = 1)
y_train = df_train['Score']
y_test = df_test['Score']

input_shape = [X_train.shape[1]]

In [28]:
modek = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape = input_shape),
    layers.Dropout(0.3),
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1)
])

In [27]:
model.compile(
    optimizer='adam',
    loss='mae'
)